In [9]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [10]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [11]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [12]:
# # create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [13]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# Access the classes attribute to get all the classes
classes = Base.classes

# reflect the tables
for class_name, class_obj in classes.items():
    print(class_name)

# View all of the classes that automap found
classes = Base.classes

measurement
station


In [14]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [15]:
# Find the most recent date in the data set.
recent_date = session.query(func.max(hawaii.date)).scalar()
print(recent_date)

NameError: name 'hawaii' is not defined

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
latest_date = session.query(hawaii.date).order_by(hawaii.date.desc()).first()[0]
one_year_ago = latest_date - pd.Timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
precipitation_data = session.query(hawaii.date, hawaii.prcp)\
                            .filter(hawaii.date >= one_year_ago)\
                            .order_by(hawaii.date)\
                            .all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
hawaii_df = pd.DataFrame(precipitation_data, columns=['Date', 'Precipitation'])

# Sort the dataframe by date
hawaii_df = hawaii_df.sort_values('Date')


# Use Pandas Plotting with Matplotlib to plot the data
df.plot(x='Date', y='Precipitation', figsize=(10, 6))
plt.title('Precipitation Over the Last 12 Months')
plt.xlabel('Date')
plt.ylabel('Precipitation (inches)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
# Assuming 'df' is your DataFrame containing the precipitation data
summary_statistics = hawaii_df['Precipitation'].describe()

print(summary_statistics)


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the datase
total_stations = session.query(func.count(hawaii.station_id)).scalar()

print(total_stations)


In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
most_active_stations = session.query(hawaii.station, func.count(hawaii.station))\
                              .group_by(hawaii.station)\
                              .order_by(func.count(hawaii.station).desc())\
                              .all()
print(most_active_stations)

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = session.query(hawaii.station, func.count(hawaii.station))\
                              .group_by(hawaii.station)\
                              .order_by(func.count(hawaii.station).desc())\
                              .first()[0]

temperature_stats = session.query(func.min(hawaii.tobs), func.avg(hawaii.tobs), func.max(hawaii.tobs))\
                            .filter(hawaii.station == most_active_station)\
                            .first()

print(temperature_stats)


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
# Step 1: Calculate the date one year ago from the most recent date in the dataset
latest_date = session.query(func.max(hawaii.date)).scalar()
one_year_ago = latest_date - pd.Timedelta(days=365)

# Step 2: Perform a query to retrieve temperature observations for the most active station from the last 12 months
temperature_observations = session.query(hawaii.tobs)\
                                  .filter(hawaii.station == most_active_station)\
                                  .filter(hawaii.date >= one_year_ago)\
                                  .all()

# Step 3: Save the query results into a Pandas DataFrame
hawaii_tobs_df = pd.DataFrame(temperature_observations, columns=['Temperature'])

# Step 4: Plot the histogram of temperature observations
plt.hist(df['Temperature'], bins=12, edgecolor='black')
plt.title('Temperature Observations for the Last 12 Months')
plt.xlabel('Temperature (°F)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


# Close Session

In [ ]:
# Close Session
session.close()